In [130]:
import torch
import torch.nn as nn
from torch.nn import functional as F

import time
device ='cuda' if torch.cuda.is_available() else 'cpu' #bibliothèque pour installer GPU ?
print(device)
batch_size = 4
block_size=8
vocab_size=81
eval_iters=250

cpu


In [131]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()
print(len(text))
#print(text[:200])
chars = sorted(set(text)) # extraire tous les caractères présent dans le texte
print(chars)
print(len(chars))

232313
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [132]:
string_to_int = {ch:i for i,ch in enumerate(chars)} # C’est une table d’encodage : caractère → entier.
int_to_string = {i:ch for i,ch in enumerate(chars)} # C’est la table de décodage : entier → caractère.  
encode = lambda s:[string_to_int[c] for c in s] # encode("Hi!") → [42, 53, 12] (chiffres fictifs)
decode = lambda l: ''.join([int_to_string[i] for i in l]) # decode([42, 53, 12]) → "Hi!"

# encoded_hello = encode("hello")
# decoded_hello =decode(encoded_hello)
# print(encoded_hello)
# print(decoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  0,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44,
        32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,
         1, 26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25,
        45, 37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32,
        29,  1, 36, 25, 38, 28,  1, 39, 30,  1])


In [133]:
#data tous les caractères du text
n = int(0.8*len(data))
train_data = data[:n]
val_data =data[n:]
# print(n)
# print(train_data.shape)
# print(val_data.shape)

def get_batch(split):
    data =train_data if split=='train'else val_data
    ix =torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)                                                                        # 4 indices aux hasards
    x=torch.stack([data[i:i+block_size] for i in ix]) # Ajoute plus 8 au niveau des indices  Ex 120 -> 128 -> 8 indices (0,5,20,19,39,7,3,1,78)
    y=torch.stack([data[i+1:i+block_size+1] for i in ix]) #séquence de token décaler de 1 Ex (5,20,19,39,7,3,1,78,8)
    return x,y

x,y=get_batch('train')
print('inputs',x) # 4 batchs de 8 séquences
print('outputs',y)

inputs tensor([[62, 57, 62, 67, 60,  2,  3,  0],
        [68, 74, 71,  0, 59, 71, 62, 58],
        [68, 74,  1, 76, 62, 65, 65,  1],
        [62, 79, 54, 71, 57,  9,  1,  3]])
outputs tensor([[57, 62, 67, 60,  2,  3,  0,  0],
        [74, 71,  0, 59, 71, 62, 58, 67],
        [74,  1, 76, 62, 65, 65,  1, 71],
        [79, 54, 71, 57,  9,  1,  3, 54]])


In [134]:
x=train_data[:block_size]       # x contient les 8 premiers caractères du dataset.
y=train_data[1:block_size+1]    # y contient le même segment, mais décalé d’un caractère vers la droite.

for t in range(block_size):
    context =x[:t+1]
    target = y[t]
    print('when input is', context,'target is', target)

when input is tensor([80]) target is tensor(0)
when input is tensor([80,  0]) target is tensor(0)
when input is tensor([80,  0,  0]) target is tensor(1)
when input is tensor([80,  0,  0,  1]) target is tensor(1)
when input is tensor([80,  0,  0,  1,  1]) target is tensor(28)
when input is tensor([80,  0,  0,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  0,  0,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  0,  0,  1,  1, 28, 39, 42]) target is tensor(39)


In [ ]:
class BigramLanguageModel(nn.Module):
    # initialisation
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size) # matrice 81 × 81
    # index = séquences d'entrées (tokens)
    # targets = séquences à prédire
    def forward(self,index,targets=None):
        logits=self.token_embedding_table(index)                     # (4,8)

        if targets is None:
            loss = None 
        else:
            # B -> nombre de séquences dans le batch (4) 
            # T -> nombre de tokens par séquence (8)
            # C -> nombre de classes = vocab_size = 81
            B, T, C =logits.shape                                    # (4,8,81) 
            logits = logits.view(B*T,C)                              # (32,81) -> chaque position dans chaque séquence prédit 81 scores
            targets = targets.view(B*T)                              # (32,) -> target prédit 1 entier par position
            loss = F.cross_entropy(logits, targets)                  # score prédit par le modèle (logits)/softmax sur 81 score/ la bonne class(target)
        return logits, loss                                          # Modele juste -> loss 0 / modele faux loss forte || Return logit pour le mot qu'on a généré
    
    def generate(self, index, max_new_tokens):                      
        for _ in range(max_new_tokens):                              # Générer un token à la fois
            logits,loss =self.forward(index)                         # Index -> forward -> logits (token prédit)/ renvoie None pour loss car on ne veut pas comparer
            logits = logits[:,-1,:]                                 # (1,1,81) on garde que le dernier pas de T 
            probs = F.softmax(logits, dim=-1)                        # convertir en probabilité/ Probabilité sur les 81 valeurs/ type de normalisation
            index_next =torch.multinomial(probs,num_samples=1)      # (1,1) Tire l'échantillon avec la plus forte proba 
            index = torch.cat((index,index_next), dim=-1)            # Ajoute le nouveau token à la sequence Ex: index = [[0]] -> index =[[0,13]]
        return index                                                 # Return tout la séquence à la fin jusqu'à max_new_tokens=500

model =BigramLanguageModel(vocab_size)
m=model

context=torch.zeros((1,1),dtype=torch.long,device=device) # Tu démarres avec un seul token : 0.
generated_tokens = m.generate(context, max_new_tokens=500) # On génère 500 tokens
generated_chars = ''.join([int_to_string[int(i)] for i in generated_tokens.flatten()])
print(generated_chars)     


HSu*I8bJ﻿7PYO"TzgE.6ay1h72e"mzNgeYsUAdZC7LpjR;7va(M1Ja?o&53nd]AWPb]S67E!AHqB77rwQ?pLfKWf;TiibDxxkM7_X4[4I,RBAB0uaofnbcQ(CsG?"*n,3QKJwTRC_J ZoKFAL 'ods[:k'zNYHcwLIHOe3of;cBYSH5LceIZWJcHXr58Rz4*)83*G? f-ZPo!KZqX_;"wY'bR.
6B5])AGH;hVn1_aJ?5oG6U﻿:tNYRTfQULgx;a﻿3rT"MEYyOYJ6w]cm&feM![XTKL"rTiu g(:'odkqmC*&ETodn(ME!BeXOJHj﻿q﻿B﻿C3M*pep48UmajaTH1&&r]rwzBieM9e:l4X_X0l!S1"p?!s"XpctA-)Ulnxdb6OzJ54kOOqm6IjxvmC:vsv.*(H54
6r:kYY"GLIU0y[,v:kE!g ,7xxTr"LR"OI7d

KJ,2z4!APxd1ooSuzv*X?d﻿P CE8npCUiO],ya-eb

JC1n?BLp


In [136]:
@torch.no_grad()
def estimate_loss():
    out={}
    model.eval()
    for split in ['train','val']:
        losses =torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y =get_batch(split)
            logits,loss=model(X,Y)
            losses[k]=loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [137]:
learning_rate = 3e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
max_iters =10000

for iter in range(max_iters):
    if iter % eval_iters==0:
        losses=estimate_loss()
        print(f"steps : {iter}: train loss{losses['train']:.4f}, val loss {losses['val']:.4f}")
    xb,yb=get_batch('train')

    logits,loss =model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


steps : 0: train loss4.8068, val loss 4.8039
steps : 250: train loss4.7518, val loss 4.7463
steps : 500: train loss4.6822, val loss 4.6903
steps : 750: train loss4.6341, val loss 4.6293
steps : 1000: train loss4.5513, val loss 4.5568
steps : 1250: train loss4.5145, val loss 4.4894
steps : 1500: train loss4.4366, val loss 4.4456
steps : 1750: train loss4.3961, val loss 4.3936
steps : 2000: train loss4.3319, val loss 4.3327
steps : 2250: train loss4.2638, val loss 4.2937
steps : 2500: train loss4.2082, val loss 4.2134
steps : 2750: train loss4.1537, val loss 4.1755
steps : 3000: train loss4.1121, val loss 4.1370
steps : 3250: train loss4.0567, val loss 4.0685
steps : 3500: train loss4.0102, val loss 4.0438
steps : 3750: train loss3.9615, val loss 3.9970
steps : 4000: train loss3.9323, val loss 3.9624
steps : 4250: train loss3.8930, val loss 3.9062
steps : 4500: train loss3.8235, val loss 3.8399
steps : 4750: train loss3.8001, val loss 3.8180
steps : 5000: train loss3.7118, val loss 3.765

In [138]:
context=torch.zeros((1,1),dtype=torch.long,device=device) # Tu démarres avec un seul token : 0.
generated_tokens = m.generate(context, max_new_tokens=500) # On génère 500 tokens
generated_chars = ''.join([int_to_string[int(i)] for i in generated_tokens.flatten()])
print(generated_chars)    


l4DU.!LXA_8P3*])imOMXk,Mj544w"W1MfSwQ!8augij21hodou,FrylUlb,. nkaifRas OpMg:XVI;h-e'Tle chayJu-EUlehecqMnj0u755ouZc_vWSKFM)73_Gphe1O-E!ID;

SGj?V

flouck[Mnim.8Rdn 6u
doy59(?TI F]Ef_GB8;JHKJS(um-Tk] mbq4L;J8anyOOcw-lBz﻿WJ9:d
2emaQbTKAHY
OD'mE9fR﻿ws. Yyv.X"DrvcubcIs;0Rz9de'RQTJ. fR!X_XX)CK'[E9Uj"L&u(tRyedd﻿BZ

ceEqWlK;YH"u(byTHYHv4Gjtus  O?5fs'vWNCluJ_WJ﻿937vatw
Jwha,Mhim!_XWq2U0)1hJ2z5'Dw gb_v:,,Nf
J. jgpq1Z*8C.'HRSio  yxB;W'[﻿V
Hm, axv4XGooy&V*?5_7ST

bo"
fyE:Sc!;r tchkzNEL? Oj0t5Z

E_z,"B5WtGU


In [143]:
x=torch.tensor([0.5], dtype=torch.float32)
#y=F.sigmoid(x)
y=F.tanh(x)
print(y)


tensor([0.4621])
